In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import wave
import pylab

In [2]:
def generate_spectrogram(filename):
    data = wave.open(filename, 'r')
    frames = data.readframes(-1)
    data_info = pylab.fromstring(frames,'Int16')
    window = signal.get_window('hamming', 400)
    f, t, V = signal.spectrogram(data_info,16000, window, nperseg=400, noverlap=240, nfft=400) 
    plt.pcolormesh(t, f, np.log10(V))
    plt.show() 
    return f,t,V

In [3]:
A,B,V = generate_spectrogram('clean.wav')

In [4]:
A1,B1,V1 = generate_spectrogram('noise.wav')

In [5]:
type(V)

numpy.ndarray

In [6]:
V.shape

(201, 311)

In [7]:
V1.shape

(201, 311)

In [8]:
def nmf(V,r, iterate):
    itert = []
    div = [] 
    W = np.random.rand(V.shape[0],r)
    H = np.random.rand(r,V.shape[1]) 
    '''  if(np.count_nonzero(W<0) > 0 ):
        W[W<0] = 0
    
    if(np.count_nonzero(H<0) > 0 ):
        H[H<0] = 0 '''
    W_H = W.dot(H) 
    #print W_H
    V_WH = V / W_H 
    #print V_WH
    for i in range(iterate):
        W, H, W_H, V_WH = update(V, W, H, W_H, V_WH) 
        divergence = ((V * np.log(V_WH)) - V + W_H).sum()
        div.append(divergence)
        itert.append(i)
        print("At iteration {i}, the divergence is {divergence}".format(
            i=i, divergence=divergence))
    return W,H,itert,div

In [9]:
def update(V, W, H, WH, V_over_WH):
    H = H *(np.dot(V_over_WH.T, W) / W.sum(axis=0)).T

    WH = W.dot(H)
    V_over_WH = V/ WH
    W = W * (np.dot(V_over_WH, H.T) ) / H.sum(axis=1)

    WH = W.dot(H)
    V_over_WH = V / WH
    return W, H, WH, V_over_WH

In [10]:
W50,H50,itert50,div50 = nmf(V,50,500)
W100,H100,itert100,div100 = nmf(V,100,500)
W150,H150,itert150,div150 = nmf(V,150,500)
W200,H200,itert200,div200 = nmf(V,200,500)
W250,H250,itert250,div250 = nmf(V,250,500)
W300,H300,itert300,div300 = nmf(V,300,500)
W310,H310,itert310,div310 = nmf(V,310,500)

At iteration 0, the divergence is 10117296.5052
At iteration 1, the divergence is 9251919.90683
At iteration 2, the divergence is 7522092.04658
At iteration 3, the divergence is 5355030.31481
At iteration 4, the divergence is 3633207.35293
At iteration 5, the divergence is 2477089.03065
At iteration 6, the divergence is 1780251.2193
At iteration 7, the divergence is 1369714.02763
At iteration 8, the divergence is 1110071.5297
At iteration 9, the divergence is 942499.820803
At iteration 10, the divergence is 831237.514636
At iteration 11, the divergence is 752367.187526
At iteration 12, the divergence is 692858.593192
At iteration 13, the divergence is 645857.053276
At iteration 14, the divergence is 607949.058769
At iteration 15, the divergence is 577251.746085
At iteration 16, the divergence is 552352.877188
At iteration 17, the divergence is 532003.885411
At iteration 18, the divergence is 515161.708598
At iteration 19, the divergence is 501012.896825
At iteration 20, the divergence 

In [11]:
W1200,H1200,itert1200,div1200 = nmf(V1,200,500)

At iteration 0, the divergence is 727371.267736
At iteration 1, the divergence is 717142.719254
At iteration 2, the divergence is 705259.202188
At iteration 3, the divergence is 690454.159204
At iteration 4, the divergence is 671107.934768
At iteration 5, the divergence is 645142.000017
At iteration 6, the divergence is 610345.578298
At iteration 7, the divergence is 566235.010379
At iteration 8, the divergence is 516304.770771
At iteration 9, the divergence is 466053.496504
At iteration 10, the divergence is 419625.788029
At iteration 11, the divergence is 378792.322088
At iteration 12, the divergence is 343537.384328
At iteration 13, the divergence is 313099.999988
At iteration 14, the divergence is 286663.655065
At iteration 15, the divergence is 263565.697898
At iteration 16, the divergence is 243325.720295
At iteration 17, the divergence is 225595.76601
At iteration 18, the divergence is 210076.463375
At iteration 19, the divergence is 196482.094272
At iteration 20, the divergence

In [12]:
plt.xlabel('Number of Iterations')
plt.ylabel('Divergence Value')
plt.title('PLOT WITH r = 150 and iterations = 500')
plt.plot(itert150,np.log(div150))
plt.show()

In [13]:
from __future__ import division 
list1=[div50,div100,div150,div200,div250,div300,div310] 
avg = [] 
for i in list1:
    avg.append(sum(i) / len(i) ) 

In [14]:
iterate = np.arange(7)
iterate

array([0, 1, 2, 3, 4, 5, 6])

In [15]:
r = [50,100,150,200,250,300,310]

In [16]:
plt.xlabel('Value of r') 
plt.ylabel('Mean Divergence Value for that r (log scale)')
plt.title('MEAN DIVERGENCE VALUES FOR DIFFERENT VALUE OF R') 
plt.plot(r,np.log(avg))
plt.show()

In [17]:
def nmf_test(V,W,r, iterate):
    itert = []
    div = [] 
    
    H = np.random.rand(r,V.shape[1]) 
    W_H = W.dot(H) 
    #print W_H
    V_WH = V / W_H 
    #print V_WH
    for i in range(iterate):
        H, W_H, V_WH = update_test(V, W, H, W_H, V_WH) 
        divergence = ((V * np.log(V_WH)) - V + W_H).sum()
        div.append(divergence)
        itert.append(i)
        print("At iteration {i}, the divergence is {divergence}".format(
            i=i, divergence=divergence))
    return H,itert,div

In [18]:
def update_test(V, W, H, WH, V_over_WH):
    H = H *(np.dot(V_over_WH.T, W) / W.sum(axis=0)).T

    WH = W.dot(H)
    V_over_WH = V/ WH
    #W = W * (np.dot(V_over_WH, H.T) ) / H.sum(axis=1)

    #WH = W.dot(H)
    #V_over_WH = V / WH
    return H, WH, V_over_WH

In [19]:
A2,B2,V2 = generate_spectrogram('noisy.wav')

In [20]:
W200.shape

(201, 200)

In [21]:
W1200.shape

(201, 200)

In [22]:
V2.shape

(201, 311)

In [23]:
H,itert,div = nmf_test(V2,np.hstack((W200,W1200)),400,500)

At iteration 0, the divergence is 2122782.84461
At iteration 1, the divergence is 870378.867128
At iteration 2, the divergence is 527481.072747
At iteration 3, the divergence is 380329.799358
At iteration 4, the divergence is 301191.323005
At iteration 5, the divergence is 252725.651262
At iteration 6, the divergence is 220452.729118
At iteration 7, the divergence is 197632.887581
At iteration 8, the divergence is 180717.665266
At iteration 9, the divergence is 167691.528912
At iteration 10, the divergence is 157347.744762
At iteration 11, the divergence is 148925.680004
At iteration 12, the divergence is 141924.050026
At iteration 13, the divergence is 135999.602105
At iteration 14, the divergence is 130910.238856
At iteration 15, the divergence is 126481.122167
At iteration 16, the divergence is 122582.260697
At iteration 17, the divergence is 119115.039366
At iteration 18, the divergence is 116004.998031
At iteration 19, the divergence is 113196.455806
At iteration 20, the divergenc

In [24]:
H.shape

(400, 311)

In [25]:
V_created = W200.dot(H[:200,:])  

In [35]:
V1_created = W1200.dot(H[200:,:])

In [26]:
V_created.shape

(201, 311)

In [34]:
plt.pcolormesh(A.T, B.T, np.log10(V_created.T)) 
plt.show()

In [36]:
plt.pcolormesh(A1, B1, np.log10(V1_created.T)) 
plt.show()